# XMAGIX

1. [Loading Library, Configuration Files and Other Stuff](#load)
2. [Starting System and Initializing Parameters](#init)
3. [Setting and Getting Parameters](#setget)
    * [Get all currently set acquisition values and board information](#getvals)
        * [Board Information](#boardinfo)
        * [Acquisition values](#getacquisition)
        * [Set Acquisition Values](#setacquisition)
    * [Widgets](#widgets)
        * [Peaking Time, Bin Width and MCA-Channels](#peaking)
        * [Thresholds](#thresh)
        * [Gain](#gain)
4. [X-Ray Source Control](#source)
    * [Setting HV and Filament Current](#write)
    * [Reading back HV and Filament Current](#read)
    * [Enable/Disable](#enabledisable)
5. [Starting and Stopping a Run](#startstop)
    * [Pick preset type](#pick)
    * [Fixed Realtime Run](#fixed)
6. [Pull Data and Plot](#pull)
    * [Happy with your Spectrum?](#happy)

☝️ Click [🔝](#XMAGIX) to jump back to the top.

# Loading Library, Configuration Files and Other Stuff [🔝](#XMAGIX) <a id="load"></a>

In [ ]:
import os
from ctypes import *
import ipywidgets as widgets
from IPython.display import display
from IPython.core.display import SVG
from rich.jupyter import print
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = "svg" # crisper plots
import numpy as np
import pandas as pd
# import lmfit as lm

from xmagix import XMagix
from thetube import Tube

tube = Tube()

cwd = os.getcwd()
xmagix = XMagix("lib/libhandel.so")
inifile = cwd + "/microdxp_usb2.ini"
configfile = cwd + "/config.ini"

In [ ]:
#!~/klippy-env/bin/python ~/klipper/klippy/klippy.py ~/printer_data/config/printer.cfg -a /tmp/klippy_uds -l /tmp/klippy.log

# Starting System and Initializing Parameters [🔝](#XMAGIX) <a id="init"></a>

Available Log Levels:
```
1:  MD_ERROR
2:  MD_WARNING
3:  MD_INFO
4:  MD_DEBUG
```

Issue ``tail -f /tmp/xmagix.log`` in a separate shell for live viewing log output.

In [ ]:
xmagix.setLogging(4)
xmagix.exit()
xmagix.init(inifile)
xmagix.startSystem()

# Setting and Getting Parameters [🔝](#XMAGIX) <a id="setget"></a>

## Get all currently set acquisition values and board information [🔝](#XMAGIX) <a id="getvals"></a>

### Board Information [🔝](#XMAGIX) <a id="boardinfo"></a>

In [ ]:
binfo = xmagix.getBoardInformation()
print(binfo)

glow, ghi, gexpo = binfo["Gain (mantissa low byte)"], binfo["Gain (mantissa high byte)"], binfo["Gain (exponent)"]
mantissa = ghi << 8 | glow
gain = (mantissa/32768)*2**gexpo
print(f"Gain: {gain}")

### Acquisition values [🔝](#XMAGIX) <a id="getacquisition"></a>

In [ ]:
values = xmagix.getAcquisitionValues(name="all")
values = pd.DataFrame(values.items(), columns=["Parameter", "Value"])
# print(values.sort_values(by=["Parameter"]))
print(values)

### Set Acquisition Values [🔝](#XMAGIX) <a id="setacquisition"></a>

In [ ]:
params = {
    "parset":     2.400000, # 2.000000
    # "genset":     0.000000,
    # "fippi":     0.000000,
    "clock_speed":    40.000000,
    "energy_gap_time":     0.300000, # 0.3
    "trigger_peak_time":     0.050000,
    "trigger_gap_time":     0.000000,
    "baseline_length":     512.000000, # 512.000000
    "trigger_threshold":   150.000000, # 100.000000
    "baseline_threshold":   120.000000, # 120.000000
    "energy_threshold":     0.000000, # 0.000000
    "peak_interval_offset":     0.500000,
    "peak_sample_offset":     0.000000,
    "max_width":     0.400000,
    "peak_mode":     0.000000,
    "peak_interval":     0.500000,
    "peak_sample":     0.000000,
    "polarity":     1.000000,
    "preamp_value":     1.000000, # reset interval in us
    "gain":     4.484848, # 3.639965
    "gain_trim":     1.000000, # 1.018982
    "preset_type":     0.000000,
    "preset_value":     0.000000,
    "number_mca_channels":  8192.000000,
    "mca_bin_width":     1.000000,
    "bytes_per_bin":     3.000000,
    "adc_trace_wait":     0.025000,
    "auto_adjust_offset":     1.000000,
    "number_of_scas":     0.000000,
}
xmagix.setParams(params)

## Widgets [🔝](#XMAGIX) <a id="widgets"></a>

### Peaking Time, Bin Width and MCA-Channels [🔝](#XMAGIX) <a id="peaking"></a>

Get available peaking times

In [ ]:
ccurrentPeakingTimes = xmagix.getNumberOfPeakingTimes()
currentPeakingTimes = np.ctypeslib.as_array(ccurrentPeakingTimes)
print(currentPeakingTimes)

In [ ]:
idict = {"manual": True, "manual_name":"Set"}
pt = widgets.interactive(xmagix.setAcquisitionValues, idict, name=widgets.fixed("parset"), value=widgets.Dropdown(options=currentPeakingTimes.round(2).tolist(), value=2.4, description="Peaking Time"))
bw = widgets.interactive(xmagix.setAcquisitionValues, idict, name=widgets.fixed("mca_bin_width"), value=widgets.FloatText(value=1, description="Bin Width"))
nMCA = widgets.interactive(xmagix.setAcquisitionValues, idict, name=widgets.fixed("number_mca_channels"), value=widgets.FloatText(value=8192, description="No. MCA"))
ui1 = widgets.HBox([pt, bw, nMCA])
ui1

### Thresholds [🔝](#XMAGIX) <a id="thresh"></a>

```
Trigger: 0-4095
Baseline: 0-4095
Energy: 0-4095
```

In [ ]:
th_trigger = widgets.interactive(xmagix.setAcquisitionValues, idict, name=widgets.fixed("trigger_threshold"), value=widgets.FloatText(value=150, description="Trigger Thr."))
th_baseline = widgets.interactive(xmagix.setAcquisitionValues, idict, name=widgets.fixed("baseline_threshold"), value=widgets.FloatText(value=120, description="Baseline Thr."))
th_energy = widgets.interactive(xmagix.setAcquisitionValues, idict, name=widgets.fixed("energy_threshold"), value=widgets.FloatText(value=0, description="Energy Thr."))
ui2 = widgets.HBox([th_trigger, th_baseline, th_energy])
ui2

### Gain [🔝](#XMAGIX) <a id="gain"></a>

$Base Gain = \frac{1184}{DynRange[keV] \cdot Preamp Gain [\frac{mV}{keV}]}$

Where DynRange is the maximum expected photon energy i.e. the set acceleration voltage.

$Preamp Gain \approx 6.6 \frac{mV}{keV}$ is taken from the Technical Report.

```
Gain: 1-100 (4.4994)
Gain Trim: 0.5-2.0 (-1)
Gap Time: (0.3)
```

In [ ]:
gain = widgets.interactive(xmagix.setAcquisitionValues, idict, name=widgets.fixed("gain"), value=widgets.FloatText(value=4.4994, description="Gain"))
gain_trim = widgets.interactive(xmagix.setAcquisitionValues, idict, name=widgets.fixed("gain_trim"), value=widgets.FloatText(value=1, description="Gain Trim"))
gap_time = widgets.interactive(xmagix.setAcquisitionValues, idict, name=widgets.fixed("trigger_gap_time"), value=widgets.FloatText(value=.3, description="Gap Time"))
ui3 = widgets.HBox([gain, gain_trim, gap_time])
ui3

# X-Ray Source Control [🔝](#XMAGIX) <a id="source"></a>

## Setting HV and Filament Current [🔝](#XMAGIX) <a id="write"></a>

In [ ]:
tube.setHV(20)
tube.setI(600)

## Reading back HV and Filament Current [🔝](#XMAGIX) <a id="read"></a>

In [ ]:
HV = tube.read(0)
I = tube.read(1)
print(f"{HV[1]:.1f} kV ({HV[0]})\n{I[1]:.1f} uA ({I[0]})")

## Enable/Disable [🔝](#XMAGIX) <a id="enabledisable"></a>

In [ ]:
tube.enable()

In [ ]:
tube.disable()

# Starting and Stopping a Run [🔝](#XMAGIX) <a id="startstop"></a>

## Pick preset type [🔝](#XMAGIX) <a id="pick"></a>

```
0.0: NONE
1.0: FIXED REAL
2.0: FIXED LIVE
3.0: FIXED EVENTS
4.0: FIXED TRIGGERS
```

In [ ]:
xmagix.setAcquisitionValues("preset_type", 0.0)
xmagix.startRun(clearMca=True)

In [ ]:
xmagix.stopRun()

## Fixed Realtime Run [🔝](#XMAGIX) <a id="fixedreal"></a>

In [ ]:
tube.enable()
xmagix.fixedRealtimeRun(realtime=30, clearMca=True)
tube.disable()

# Pull Data and Plot [🔝](#XMAGIX) <a id="pull"></a>

In [ ]:
mca = xmagix.pullMcaData()

In [ ]:
plt.style.use("default")
plt.style.use("dark_background")
cm = 1/2.54 # inch to cm
size = 60

Fe = [6.40384, 6.39084, 7.05798]
Co = [6.93, 6.92, 7.65]
W = [59.32, 57.98, 67.24, 8.4, 8.34, 9.67, 9.96, 11.29, 1.78]
Cu = [8.05, 8.03, 8.91]
Zn = [8.64, 8.62, 9.57]

titles = [f"nMCA {len(mca)}", "Energy / keV", "Counts"]
fig, ax = plt.subplots(1, 1, figsize=(40*cm, 25*(9/16)*cm))
# x = np.arange(0, len(mca), 1)
x = np.linspace(0, 40, len(mca))
y = np.vstack([mca])
for line in W:
    ax.axvline(x=line, color="red", ls="-", lw=1)
for line in Cu:
    ax.axvline(x=line, color="orange", ls=":", lw=1)
for line in Zn:
    ax.axvline(x=line, color="grey", ls=":", lw=1)
for line in Fe:
    ax.axvline(x=line, color="grey", ls="--", lw=1)
ax.stackplot(x, y, labels=["Steel"])
ax.set(xlabel=titles[1], ylabel=titles[2], xlim=[x.min(), x.max()])
ax.set_yscale("log")
ax.legend()
plt.tight_layout()

if not os.path.exists("./tmp/"):
    os.makedirs("./tmp/") 
plt.savefig("./tmp/plot.svg")
plt.savefig("./tmp/plot.png")
# SVG("./tmp/plot.svg")

## Happy with your Spectrum? [🔝](#XMAGIX) <a id="happy"></a>

Run below cell and input a name when prompted.

In [ ]:
csvPath = input()
# csvPath = "Pb_XRF"
sv_values = xmagix.getAcquisitionValues(name="all")
sv_values = pd.DataFrame(sv_values.items(), columns=["Parameter", "Value"])
if not os.path.exists(f"./{csvPath}"):
    os.makedirs(f"./{csvPath}")
os.rename("./tmp/plot.svg", f"./{csvPath}/{csvPath}.svg")
os.rename("./tmp/plot.png", f"./{csvPath}/{csvPath}.png")
with open(f"./{csvPath}/{csvPath}.csv", "x") as f:
    sv_values.to_csv(f, sep=";", index=False)
    mcapd = pd.DataFrame(mca)
    f.write("BEGIN_OF_DATA")
    mcapd.to_csv(f, sep=";")
    f.write("END_OF_DATA")